In [100]:
import numpy as np
import random

#--------------- Scenario Configuration

# Initialize configuration variables.
N_list = [50, 100, 500, 1000, 5000, 10000]
Qf_list = [1, 5, 10, 15]
sig_list = [0.01, 0.05, 0.1, 0.5, 1.0]

o2AvgTestMSEs = []
o10AvgTestMSEs = []

# Get average test MSE for every configuration of N, Qf, and sig.
for N in N_list:
    for Qf in Qf_list:
        for sig in sig_list:

            # Repeat everything below multiple times (1000) in order to get expected out-of-sample error for this Q_f, N, sig scenario.
            o2TestMSEs = []
            o10TestMSEs = []

            for i in range(0, 1000):

                #--------------- Data and Parameter Generation

                # Generate the data.
                D = np.random.uniform(-1, 1.00001, N)

                # Generate the errors from a standard normal distribution.
                eps = np.random.normal(0.0, 1.0, N)

                # Generate the target Legendre polynomial coefficients from a normal distribution.
                a = np.random.normal(0.0, 1.0, Qf)

                #--------------- Targets

                # Set target function f(x) as sum of Legendre polynomials to order Qf with above coefficients.
                f = np.polynomial.legendre.Legendre(coef=a)

                # Rescale coefficients such that the expectation value of f^2 is 1.
                # Average of a function is 1/(b-a) times its integral from b to a.
                integ = f.integ(m=1)
                avg = 0.5*(np.polynomial.polynomial.polyval(1, integ.coef) - np.polynomial.polynomial.polyval(-1, integ.coef))
                # Rescaling coefficients by 1/avg produces an expectation value of f^2 that is 1.
                a = a/avg
                f = np.polynomial.legendre.Legendre(coef=a)
                integ = f.integ(m=1)
                avg = 0.5*(np.polynomial.polynomial.polyval(1, integ.coef) - np.polynomial.polynomial.polyval(-1, integ.coef))

                # Calculate target outputs.
                y_target = []
                for i in range(0, len(D)):
                    # Evaluate target function on point.
                    f_x = np.polynomial.polynomial.polyval(D[i], f.coef)
                    # Apply random noise.
                    noise = sig * eps[i]
                    y_target.append(f_x + noise)

                #--------------- Data Split

                # Split data into training and testing (70/30).
                numTrain = int(0.7*len(D))
                # The first list in D_train is x values; the second is targets.
                D_train = [[], []]
                # D_test has the same structure.
                D_test = [[], []]
                for i in range(0, numTrain):
                    index = random.randint(0,len(D)-1)
                    D_train[0].append(D[index])
                    D_train[1].append(y_target[index])
                    D = np.delete(D, index)

                for i in range(0, len(D)):
                    index = random.randint(0,len(D)-1)
                    D_test[0].append(D[index])
                    D_test[1].append(y_target[index])
                    D = np.delete(D, index)

                #--------------- Fitting and Evaluation of Hypotheses

                # Train second-order polynomial on training data.
                order2Fit = np.polynomial.polynomial.Polynomial.fit(x=D_train[0], y=D_train[1], deg=2)

                # Train tenth-order polynomial on training data.
                order10Fit = np.polynomial.polynomial.Polynomial.fit(x=D_train[0], y=D_train[1], deg=10)

                # Evaluate fits on test data and calculate test errors.
                o2TestMSE = 0
                o10TestMSE = 0
                for i in range(0, len(D_test[0])):
                    o2TestPred = np.polynomial.polynomial.polyval(D_test[0][i], order2Fit.coef)
                    o2SqdErr = np.power((o2TestPred - D_test[1][i]),2)
                    o2TestMSE += float(o2SqdErr)/len(D_test[0])

                    o10TestPred = np.polynomial.polynomial.polyval(D_test[0][i], order10Fit.coef)
                    o10SqdErr = np.power((o10TestPred - D_test[1][i]),2)
                    o10TestMSE += float(o10SqdErr)/len(D_test[0])

                o2TestMSEs.append(o2TestMSE)
                o10TestMSEs.append(o10TestMSE)

            avgO2TestMSE = 0
            avgO10TestMSE = 0
            for i in range(0, len(o2TestMSEs)):
                avgO2TestMSE += float(o2TestMSEs[i])/len(o2TestMSEs)
                avgO10TestMSE += float(o10TestMSEs[i])/len(o10TestMSEs)

            o2AvgTestMSEs.append(avgO2TestMSE)
            o10AvgTestMSEs.append(avgO10TestMSE)

In [101]:
print(o2AvgTestMSEs)

[9.973266571584917e-05, 0.00254212617306974, 0.010834643871775235, 0.2606997545237652, 1.0161649835657054, 429.1213363058598, 195.78103219068907, 10009.270103607483, 217.61177923352963, 369617.9024039437, 252.695203189809, 6744.690735783332, 2053.4011542898643, 3122.620070060361, 1998.0937569557311, 6336.100183678717, 64714.659678352364, 3146.1692263440113, 88601.99158965136, 15821.667892585881, 0.00010220795626153227, 0.002535576000326178, 0.010197798999924554, 0.26076409586775917, 1.0241058888207095, 175.89049316193197, 647.5273912832906, 3070.8717475157314, 493449.05449019215, 1337.763556309225, 362.2302149945804, 1060.3674924721156, 139696.17935628042, 321177.9126151434, 498.951966868121, 580.5817916054959, 282.2841394687083, 1357.8989060572642, 295.36647000347864, 221.06971412609838, 0.00010020258161992833, 0.002517207694479989, 0.010103126061525827, 0.25118199101595334, 1.0017614924760332, 1938.6227740584973, 35419.618440055754, 112356.92103371635, 8684.54482037487, 43557.5523302

In [102]:
print(o10AvgTestMSEs)

[0.0002588857095676121, 0.005731946212603433, 0.021373124943193853, 0.5144534896874029, 3.5795259299949875, 933.6168649755975, 592.7494605423349, 16749.977726275858, 341.88612951303634, 472965.07706119854, 582.7776657245975, 7701.7800776728745, 3025.3893490886603, 3461.6462113286116, 2092.720541618195, 6676.431168123585, 132381.82757464377, 3133.3996547467204, 108866.40626792537, 26966.941751157323, 0.0001159617150329293, 0.0028470463595659428, 0.011536996187632945, 0.2933369074629778, 1.1457961134411698, 183.7980221107287, 792.1499810618983, 3507.37125995148, 579664.8629119719, 1434.6260050813214, 451.2733805046412, 1204.4418145872207, 149004.25476804964, 444339.86486501276, 649.0957697992766, 650.2649662390172, 363.57614471622867, 2020.8216345875371, 403.8816870222983, 260.2767433108452, 0.00010248247515119942, 0.0025764151743780384, 0.010317601034849579, 0.2568567008768678, 1.0248273197613296, 1990.2134290111844, 36379.0652934515, 114346.0733677768, 8837.477747417744, 45330.48477779

In [105]:
for i in range(0, len(o2AvgTestMSEs)):
    val = o2AvgTestMSEs[i] < o10AvgTestMSEs[i]
    if (not val):
        print(i)

17
73
92
119
